# Package 2 - Query and Summary

### Download Packages

In [ ]:
# Python 3.12.9 was used to develop this code.
# standard library
import os
import json
import textwrap
import warnings
from typing import List, Optional, Tuple, Dict
from pprint import pprint
from collections import Counter
from datetime import datetime, date
import re
import time
from enum import Enum

# External Libraries
import pandas as pd
from tqdm import tqdm
from pydantic import BaseModel, PydanticDeprecatedSince20, Field
from huggingface_hub.utils import _deprecation
from concurrent.futures import ThreadPoolExecutor, as_completed
from numpy import dot
from numpy.linalg import norm

from IPython.display import display

# Langchain specific
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader
from langchain_community.vectorstores import LanceDB
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage


# LanceDB specific
import lancedb
from lancedb.pydantic import LanceModel, Vector

#Deprecation handling
import warnings
warnings.filterwarnings("ignore",
                        message=r".*'post'.*deprecated.*",
                        category=FutureWarning,
                        module=r"huggingface_hub\.utils\._deprecation"
                        )

### Connect to Table

In [ ]:
# Path to LanceDB database
lancedb_path = r"C:\Users\marcu\OneDrive\Desktop\NPS_Academics_2\Q7\LLM\Capstone\lancedb"
# Ensure the path exists
lancedb_path = os.path.join(lancedb_path, "lancedb")
# Define the LanceDB table name
db = lancedb.connect("./lancedb")
# Check if the table exists
table = db.open_table("Capstone_test_table")
# Print the number of rows in the table
print(f"LanceDB contains {table.count_rows()} rows.")

LanceDB contains 18661 rows.


### Create Functions

In [114]:
# Define the parse_filter_input function to parse the filter input
# Convert the raw filter input into a structured format
def parse_filter_input(raw_filter: Dict[str, str]) -> Dict[str, object]:
    parsed = {}

    # Define Filterable fields
    list_fields = {
        "domain": None,
        "agency": None,
        "cocom": None,
        "country": None,
        "key_words": None,
        "date_of_pub": "YEAR_INT_LIST"
    }

    for k, v in raw_filter.items():
        # skip blank filter fields
        if not v or v.strip() == "":
            continue  
        # clean up the key
        values = [x.strip() for x in v.split(",") if x.strip()]
        # Take the year from date_of_pub if it is a date
        if k in list_fields:
            if list_fields[k] == "YEAR_INT_LIST":
                parsed[k] = [int(x) for x in values]
            else:
                parsed[k] = values
        # Uses the first value in the catergory field. LLM should produce only one value
        elif k == "category":
            parsed[k] = values[0]

    return parsed

# This function checks whether a metadata row matches all user-specified filter criteria, allowing 
# for OR-style matching within list fields and case-insensitive comparisons.
def row_matches(row_metadata, filter):
    # Grab keys from the filter and row metadata
    for k, v in filter.items():
        # get the value from the row metadat
        row_value = row_metadata.get(k)
        # Special case: check if the publication year matches any of the filter years
        if k == "date_of_pub":
            if not row_value or row_value.year not in v:
                return False
        # Special case: exact (case-insensitive) string match for category
        elif k == "category":
            if not row_value or row_value.lower() != v.lower():
                return False

        else:
            # If the field is expected to be a list but isn't, it's not a match
            if not isinstance(row_value, list):
                return False

            # Normalize the metadata and filter values to lowercase sets for comparison
            row_lower = {x.lower() for x in row_value}
            filter_lower = {x.lower() for x in v}

            # OR-style logic: any match is enough
            if not row_lower & filter_lower:
                return False

    return True


def query_topic_among_country(topic: str, raw_filter: Dict[str, str], top_k: int = 10):
    """
    Filters LanceDB by metadata fields, scores relevance to the topic,
    and returns:
      - top file extensions by average score (grouped)
      - top 25 rows (chunks) by individual similarity score
    """
    # Use user-provided filter to narrow down the rows
    filter = parse_filter_input(raw_filter)
    # create a python list of rows from the LanceDB table
    rows = table.to_arrow().to_pylist()
    # Filter the rows (chunks) based on the filter criteria
    filtered = [row for row in rows if row_matches(row["metadata"], filter)]
    print(f"Found {len(filtered)} rows matching the filter criteria")

    # Embed the topic
    query_vector = tei_endpoint.embed_query(topic)

    def cosine_similarity(a, b):
        return dot(a, b) / (norm(a) * norm(b))

    # Calculate and append the similarity score for each row (chunk)
    scored = []
    for row in filtered:
        sim = cosine_similarity(query_vector, row["vector"])
        scored.append({
            "score": sim,
            "file_extension": row["metadata"].get("file_extension", ""),
            "title": row["metadata"].get("title", ""),
            "keywords": row["metadata"].get("key_words", []),
            "text": row["text"],
        })

    df = pd.DataFrame(scored)

    # Top N file extensions by average score
    top_exts = (
        df.groupby("file_extension")["score"]
        .mean()
        .sort_values(ascending=False)
        .head(top_k)
        .reset_index()
    )

    # Top 25 individual chunks
    top_chunks = df.sort_values("score", ascending=False).head(25).reset_index(drop=True)

    return top_exts, top_chunks


### Connect to Embedding Model for Query Embedding

In [41]:
endpoint_path3 = "gpu4"
info = os.popen(f"curl -s http://trac-malenia.ern.nps.edu:8080/{endpoint_path3}/info").read()
repo_id3 = "/".join(json.loads(info)['model_id'].split('/')[-2:])

print(f"Connected to embedding LLM: {repo_id3}")

tei_endpoint = HuggingFaceEndpointEmbeddings(
    model=f"http://trac-malenia.ern.nps.edu:8080/{endpoint_path3}/embed"
)


Connected to embedding LLM: nomic-ai/nomic-embed-text-v1.5


### Query Function

In [ ]:
##### For reference, Schema inputs for defined fields ########
class Domain(str, Enum):
    Land = "Land"
    Maritime = "Maritime"
    Air = "Air"
    Space ="Space"
    Cyberspace= "Cyberspace"
    Other= "Other" #create an out for the model if no domain match well!
class COCOM(str, Enum):
    INDOPACOM = "INDOPACOM"
    PACOM = "PACOM"
    CENTCOM = "CENTCOM"
    EUCOM= "EUCOM"
    AFRICOM = "AFRICOM"
    NORTHCOM ="NORTHCOM"
    SOUTHCOM = "SOUTHCOM"
    SPACECOM =  "SPACECOM"
    CYBERCOM = "CYBERCOM"
    SOCOM = "SOCOM"
    STRATCOM = "STRATCOM"
    TRANSCOM = "TRANSCOM"
    Other= "Other" #create an out for the model if no domain match well!
class Wargame_type(str, Enum):
    policy_decision = "policy"
    capbility_requirement= 'capability'
    conop_strategy = 'conop'
    process_org = "process"
    other= "other"

In [ ]:
# Filter input structure
# Not Case sensitive, but is acronym sensitive.... So USA, United States, and US are all different.
# All fields must be comma-separated strings, except for date_of_pub which is a year (int).
raw_filter = {
    "date_of_pub": "", # Now just the year (int)
    "domain": "", # List field
    "agency": "", # List field
    "cocom": "", # List field
    "country": "", # List field
    "category": "ProCeSs", # String field, accepted as-is
    "key_words": "" # List field
}

topic = "force posture in the Pacific"
top_exts, top_chunks = query_topic_among_country(topic, raw_filter, top_k=10)
display(top_exts)       # Summary of top docs by average score
display(top_chunks)     # Top 25 chunks by similarity

Found 910 rows matching the filter criteria


,file_extension,score
0,./final_docling_output/VISTA QUAKE Quick Look ...,0.536066
1,./final_docling_output/All_Hazards_After_Actio...,0.532564
2,./final_docling_output/PROSAIC TWAIN_Quick Loo...,0.484162
3,./final_docling_output/New_Mexico_Tornado_Tabl...,0.469001
4,./final_docling_output/Wargaming_Effectiveness...,0.454619
5,./final_docling_output/Kuznetsov_11_Game_Repor...,0.452629
6,./final_docling_output/Kuznetsov_13_Game_Repor...,0.449526
7,./final_docling_output/FMD_TTX_ESF11_Report.md,0.444492
8,./final_docling_output/KIBOWI_Netherlands_Army.md,0.444065
9,./final_docling_output/Battle_Simulations.md,0.440583


,id,score,file_extension,title,keywords,text
0,./final_docling_output/Wargaming_Effectiveness...,0.593796,./final_docling_output/Wargaming_Effectiveness...,Wargaming Effectiveness: Its Conceptualization...,"[battlefield, combat, posture, enemy, agents, ...",| Visualize the Battlefield | What effect does...
1,./final_docling_output/VISTA QUAKE Quick Look ...,0.583862,./final_docling_output/VISTA QUAKE Quick Look ...,VISTA QUAKE Quick Look,"[emergency response, FEMA, RRCC, disaster mana...","-  After the initial response, participants a..."
2,./final_docling_output/Kuznetsov_11_Game_Repor...,0.576512,./final_docling_output/Kuznetsov_11_Game_Repor...,Kuznetsov Naval Academy Game '11,"[Navy, task force, amphibious ships, escort sh...",- · A U. S. Navy task force comprised of three...
3,./final_docling_output/Wargaming_Effectiveness...,0.560343,./final_docling_output/Wargaming_Effectiveness...,Wargaming Effectiveness: Its Conceptualization...,"[MOPP, MSR, MTOE, NBC, OP, OPORD, OPSEC, OBJ, ...",| MOPP | Mission Oriented Protective Postur...
4,./final_docling_output/Wargaming_Effectiveness...,0.558607,./final_docling_output/Wargaming_Effectiveness...,Wargaming Effectiveness: Its Conceptualization...,"[Decision Point, DP, maneuver events, mission ...",| Decision Point (DP) Element placed on the ov...
5,./final_docling_output/Wargaming_Effectiveness...,0.557706,./final_docling_output/Wargaming_Effectiveness...,Wargaming Effectiveness: Its Conceptualization...,"[Command Posts, CPs, TAC, Main, communication,...",| Command Posts (CPs) (TAC & Main) Element (TA...
6,./final_docling_output/Wargaming_Effectiveness...,0.555697,./final_docling_output/Wargaming_Effectiveness...,Wargaming Effectiveness: Its Conceptualization...,"[NAI, monitor, civilian, population, movement,...",| NAI XX Element placed on the overlay? ...
7,./final_docling_output/VISTA QUAKE Quick Look ...,0.552250,./final_docling_output/VISTA QUAKE Quick Look ...,VISTA QUAKE Quick Look,"[earthquake, tsunami, Cascadia Subduction Zone...",![Image](VISTA QUAKE Quick Look Final_artifact...
8,./final_docling_output/VISTA QUAKE Quick Look ...,0.549864,./final_docling_output/VISTA QUAKE Quick Look ...,VISTA QUAKE Quick Look,"[economic recovery, state leadership, national...",- · Economic Recovery: State and federal parti...
9,./final_docling_output/Wargaming_Effectiveness...,0.548666,./final_docling_output/Wargaming_Effectiveness...,Wargaming Effectiveness: Its Conceptualization...,"[command post, tactical operations, task force...",References: Commander's Battle Staff Handbook...
